In [6]:
import csp_predictor as csp
import pandas as pd
import os

In [7]:
import pandas as pd
import numpy as np

def compute_agme_per_sample_mean(train_expr, pathways_csv, hidden_size=200):

    pw = pd.read_csv(pathways_csv)
    pw_copy = pw.copy()
    pw_copy[pw_copy.isnull()] = "null_gene"
    gene_set = []
    for i in range(pw_copy.shape[1]):
        gene_set += list(pw_copy.iloc[:, i])

    # 准备缺失基因列表（用于插补）
    pw_t = np.array(pw.T)
    pw_flat = pd.DataFrame(pw_t.reshape(-1)).dropna().values.reshape(-1)
    selected_genes_raw = np.array(gene_set)
    selected_genes_raw1 = selected_genes_raw.reshape(-1, hidden_size)
    selected_genes_dropna = pw_flat.reshape(-1)

    expr = train_expr
    expr_T = expr.T.copy()
    expr_T["null_gene"] = 0.0
    expr_T_copy = expr_T.copy()

    not_available = list(set(selected_genes_dropna) - set(expr.index))

    for gene_vector in selected_genes_raw1:
        avail = list(set(gene_vector) - set(not_available))
        for g in gene_vector:
            if g in not_available and g != "null_gene":
                expr_T[g] = (np.power(2, pd.DataFrame(np.log2(expr_T_copy[avail] + 1)).mean(axis=1)) - 1)


    expr_ordered = expr_T.loc[:, selected_genes_raw]

    per_sample_sum = expr_ordered.sum(axis=1).astype(float)
    return float(per_sample_sum.mean())


In [8]:
data_path = '/home/shenwanxiang/Research/aliyun_sync/COMPASS/paper/00_data/'
df_label = pd.read_pickle(os.path.join(data_path, 'ITRP.PATIENT.TABLE.ALIGN'))
df_tpm = pd.read_pickle(os.path.join(data_path, 'ITRP.TPM.TABLE'))


In [9]:
res= [] 
for cohort in df_label.cohort.unique():
    dfc = df_tpm.loc[df_label[df_label.cohort==cohort].index].T

    #dfc = np.log2(dfc + 1)
    all_gene_mean_expression_value = compute_agme_per_sample_mean(dfc, 
                                                                  pathways_csv = "./CSP_survival_model/Deep_learning_input_pathways.csv")
    
    dfp = csp.predict_df(
        expr_df=dfc,
        pathways_csv="./CSP_survival_model/Deep_learning_input_pathways.csv",
        model_pth="./CSP_survival_model/Trained_deep_learning_model.pth",
        is_tpm_format=True, #already do the transofrm
        #all_gene_mean_expression_value = all_gene_mean_expression_value
    
    )
    res.append(dfp)

37
37
37
37
37
37
37
37
37
37
37
37
37
37
37
37


In [10]:
pd.concat(res).to_csv('csp_itrp.csv')